In [2]:
import json

In [3]:
dataset = 'aishell'
recog_set = ['train', 'dev', 'test']

In [4]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/data.json'
    with open(json_name, 'r') as f, open(f'./data/{dataset}_{r}/hyp_new.trn', 'w') as hyp, open(f'./data/{dataset}_{r}/ref_new.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
            for i, h in enumerate(j['utts'][k]['output']):
                # print(j['utts'][k]['output'])
                seq = [t for t in h['rec_token'].split()]
                hyp_seq = " ".join(seq[:-1])
                hyp.write( f'{hyp_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )
                # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')

                ref_seq = h["token"]
                ref.write( f'{ref_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )

上面code跑完 執行espnet的
```sclite -r ${task_data_dir}/ref.trn -h  ${task_data_dir}/hyp.trn -i rm -o all stdout >  ${task_data_dir}/result.txt```

In [40]:
nbest_err = []
with open(f'./data/aishell_test/nbest_result.txt', 'r') as f:
    temp_err = []
    for i, line in enumerate(f):
        if ('Score' in line):
            scores = line.split()[-4:]
            temp_err.append(scores)
            if (len(temp_err) == 10):
                nbest_err.append(temp_err)
                temp_err = []

In [41]:
len(nbest_err)

7176

In [42]:
nbest_token = []
nbest_score = []
with open('./data/aishell_test/data.json', 'r') as f:
    j = json.load(f)
    for k in j['utts'].keys():
        token = []
        score = []
        for i, h in enumerate(j['utts'][k]['output']):
            token.append(h["rec_text"])
            score.append(h['score'])
        nbest_token.append(token)
        nbest_score.append(score)

print(len(nbest_token))

7176


In [43]:
dataset = []
for token, score, err in zip(nbest_token, nbest_score, nbest_err):
    temp_dict = dict()
    temp_dict['token'] = ['[CLS]' + t[:-5] + '[SEP]' for  t  in token]
    temp_dict['score'] = score
    temp_dict['err'] =  [[int(s) for s in sc] for sc in err]
    dataset.append(temp_dict)
print(len(dataset))

7176


In [49]:
with open("./data/aishell_test/dataset.json", 'w') as f:
    json.dump(dataset, f, ensure_ascii=False)